In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import scrublet as  scr
import anndata
import scanpy.external as sce
from matplotlib import pyplot as plt
%matplotlib inline

sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=300, facecolor='white')

In [ ]:
#loading matrix from SoupX
COPD_dict = {}
data_dir = "/datg/mazhuo/data/COPD/Matrix_after_SoupX/ourdata/"
for sample in ["COPD2","COPD3","COPD4","COPD5","COPD7","COPD8","COPD9","COPD10","COPD11","COPD12","COPD15","COPD16",
               "COPD21","COPD23","COPD26","COPD27","COPD28","COPD29","COPD30","COPD31","COPD32","COPD38","COPD39",
               "COPD40","COPD41","COPD45","COPD46","COPD50","COPD52","COPD53","COPD54","COPD56","COPD57","COPD58",
               "COPD64","COPD71","COPD72","COPD73","COPD77","COPD78","COPD79","COPD81","COPD82","COPD83",
               "COPD87","COPD88","COPD90","COPD91","COPD92","COPD93","COPD94","COPD95","COPD97","COPD99",
               "COPD101","COPD102","COPD103","COPD104","COPD107","COPD128",
               "HC2","HC3","HC5","HC6","HC10","HC24","HC26","HC27","HC29","HC30","HC31","HC32","HC36","HC38","HC40","HC46","HC47","HC49","HC50",
               "HS1","HS2","HS6","HS7","HS14","HS21","HS22","HS24","HS25","HS26","HS27","HS28","HS29","HS30","HS31","HS32","HS34","HS35",
               "HS41","HS43","HS44","HS46"]:
    data = sc.read_10x_mtx(data_dir + sample + "/",var_names='gene_symbols',cache = True)
    data.obs_names = [cell + "_" + sample for cell in data.obs_names]
    data.obs["sample"] = sample
    data.var['mt'] = data.var_names.str.startswith("MT-")
    data.var['hb'] = data.var_names.str.contains("^HB[^(P)]")
    data.var['ribo'] = data.var_names.str.startswith(("RPS","RPL"))
    sc.pp.calculate_qc_metrics(data, qc_vars=["mt","hb","ribo"], percent_top=None, log1p=False, inplace=True)
    data = data[data.obs.n_genes_by_counts < 8000, :]
    data = data[data.obs.n_genes_by_counts > 500, :]
    data = data[data.obs.total_counts < 50000, :]
    data = data[data.obs.total_counts > 1000, :]
    data = data[data.obs.pct_counts_mt < 15, :]
    data = data[data.obs.pct_counts_hb < 5, :]
    COPD_dict[sample] = data 

In [ ]:
#merge samples
COPD = anndata.concat(COPD_dict)
COPD.write("processed_data_py/COPD_counts.h5ad")

In [ ]:
#normalization
sc.pp.normalize_total(COPD, target_sum=1e4)
sc.pp.log1p(COPD)
#Find variable genes
sc.pp.highly_variable_genes(COPD, n_top_genes=2000, flavor = "seurat", batch_key="sample")
COPD.raw = COPD
COPD = COPD[:, COPD.var.highly_variable]
sc.pp.regress_out(COPD, ['total_counts','pct_counts_mt'])
sc.pp.scale(COPD)

In [ ]:
#PCA
sc.tl.pca(COPD, n_comps = 60)

In [ ]:
#batch correction
sc.external.pp.bbknn(COPD, batch_key = "sample")
sc.tl.umap(COPD)

In [ ]:
#clustering
sc.pp.neighbors(COPD, n_neighbors=30, n_pcs=60)
sc.tl.leiden(COPD,resolution = 0.2)

In [ ]:
sc.pl.umap(COPD, color=["leiden"],size=1, legend_loc='on data')

In [ ]:
sc.pl.dotplot(COPD,{"Epithelial":["EPCAM","AGER","SFTPC","TP63","SCGB3A2","SCGB1A1","FOXJ1","MUC5B","CHGA",],
                        "Mesenchymal":["COL1A1","PDGFRA","SERPINF1","GPC3","WIF1","ACTA2","CTHRC1","COL11A1","LAMC3","PDGFRB","MYH11","UPK3B"],
                        "Endothleial":["PECAM1","CDH5","LYVE1","CA4","FCN3","EDNRB","IGFBP3","CPE","COL15A1"],
                        "Myeloid":["PTPRC","ITGAM","CST3","LYZ","CLEC9A","CLEC10A","SCT","LAD1",
                                  "FCN1","VCAN","MTSS1","TREM2","CD68","FABP4","F13A1","MS4A2","FCGR3B"],
                        "Lymphoid":["PTPRC","MS4A1","TNFRSF17","CD3G","CD4","CD8A","CTLA4","NKG7","MCTP2"]}, 
                  groupby="leiden",dendrogram=False,standard_scale="var",cmap='Spectral_r')

In [ ]:
metadata = pd.read_csv('tables/meta101.csv', index_col=0)

In [ ]:
COPD.obs["barcode"] = COPD.obs.index
COPD.obs['UMAP1'] = COPD.obsm['X_umap'][:, 0]
COPD.obs['UMAP2'] = COPD.obsm['X_umap'][:, 1]
COPD.obs = COPD.obs.merge(metadata, left_on='sample', right_on='sample', how="left")
COPD.obs.index = COPD.obs["barcode"]

In [ ]:
#first annotation
COPD.obs["cellclass"] = ""
COPD.obs.loc[COPD.obs["leiden"].isin(["0","6","7","8"]),"cellclass"] = "Epithelial"
COPD.obs.loc[COPD.obs["leiden"].isin(["5","9","13"]),"cellclass"] = "Mesenchymal"
COPD.obs.loc[COPD.obs["leiden"].isin(["18"]),"cellclass"] = "Mesothelial"
COPD.obs.loc[COPD.obs["leiden"].isin(["3","11","15"]),"cellclass"] = "Endothelial"
COPD.obs.loc[COPD.obs["leiden"].isin(["4","10","12","14","19"]),"cellclass"] = "Myeloid"
COPD.obs.loc[COPD.obs["leiden"].isin(["1","2","16","17","20"]),"cellclass"] = "Lymphoid"

In [ ]:
sc.pl.umap(COPD, color=["cellclass"],size=1, legend_loc='on data')

In [ ]:
COPD = COPD.raw.to_adata()

In [ ]:
meso = COPD[COPD.obs["cellclass"] == "Mesothelial"].copy()
meso.obs["celltype"] = meso.obs["cellclass"]
meso_meta = meso.obs[["celltype"]]

In [ ]:
#Add second anntation from each cell class
epi_meta = pd.read_csv('tables/epi_meta.csv', index_col = "barcode")
mes_meta = pd.read_csv('tables/mes_meta.csv', index_col = "barcode")
endo_meta = pd.read_csv('tables/endo_meta.csv', index_col = "barcode")
mye_meta = pd.read_csv('tables/mye_meta.csv', index_col = "barcode")
lym_meta = pd.read_csv('tables/lym_meta.csv', index_col = "barcode")
cellmeta = pd.concat([epi_meta, mes_meta, endo_meta, mye_meta, lym_meta,meso_meta], axis=0)
COPD.obs["celltype"] = cellmeta["celltype"]

In [ ]:
#Remove dou
COPD = COPD[COPD.obs["celltype"] != "Doublets"].copy()

In [ ]:
sc.pl.umap(COPD, color=["celltype"],size=1, legend_loc='on data', legend_fontsize=3)

In [ ]:
sc.pl.umap(COPD, color=["celltype"], legend_fontsize = 8, size = 1)

In [ ]:
COPD.obs["celltype_num"] = ""
COPD.obs.loc[COPD.obs["celltype"].isin(["Immature AT1"]),"celltype_num"] = "1"
COPD.obs.loc[COPD.obs["celltype"].isin(["Mature AT1"]),"celltype_num"] = "2"
COPD.obs.loc[COPD.obs["celltype"].isin(["AT2"]),"celltype_num"] = "3"
COPD.obs.loc[COPD.obs["celltype"].isin(["TRB Secretory"]),"celltype_num"] = "4"
COPD.obs.loc[COPD.obs["celltype"].isin(["PreTB Secretory"]),"celltype_num"] = "5"
COPD.obs.loc[COPD.obs["celltype"].isin(["Basal"]),"celltype_num"] = "6"
COPD.obs.loc[COPD.obs["celltype"].isin(["Goblet"]),"celltype_num"] = "7"
COPD.obs.loc[COPD.obs["celltype"].isin(["Ciliated"]),"celltype_num"] = "8"
COPD.obs.loc[COPD.obs["celltype"].isin(["PNEC"]),"celltype_num"] = "9"

COPD.obs.loc[COPD.obs["celltype"].isin(["Adventitial fibroblast"]),"celltype_num"] = "10"
COPD.obs.loc[COPD.obs["celltype"].isin(["Alveolar fibroblast"]),"celltype_num"] = "11"
COPD.obs.loc[COPD.obs["celltype"].isin(["Myofibroblast"]),"celltype_num"] = "12"
COPD.obs.loc[COPD.obs["celltype"].isin(["Fibromyocyte"]),"celltype_num"] = "13"
COPD.obs.loc[COPD.obs["celltype"].isin(["Pericyte"]),"celltype_num"] = "14"
COPD.obs.loc[COPD.obs["celltype"].isin(["ASM"]),"celltype_num"] = "15"
COPD.obs.loc[COPD.obs["celltype"].isin(["VSM"]),"celltype_num"] = "16"

COPD.obs.loc[COPD.obs["celltype"].isin(["Mesothelial"]),"celltype_num"] = "17"

COPD.obs.loc[COPD.obs["celltype"].isin(["Lymphatic"]),"celltype_num"] = "18"
COPD.obs.loc[COPD.obs["celltype"].isin(["Aerocyte"]),"celltype_num"] = "19"
COPD.obs.loc[COPD.obs["celltype"].isin(["gCap"]),"celltype_num"] = "20"
COPD.obs.loc[COPD.obs["celltype"].isin(["Arterial"]),"celltype_num"] = "21"
COPD.obs.loc[COPD.obs["celltype"].isin(["Venous"]),"celltype_num"] = "22"

COPD.obs.loc[COPD.obs["celltype"].isin(["B"]),"celltype_num"] = "23"
COPD.obs.loc[COPD.obs["celltype"].isin(["Plasma"]),"celltype_num"] = "24"
COPD.obs.loc[COPD.obs["celltype"].isin(["Treg"]),"celltype_num"] = "25"
COPD.obs.loc[COPD.obs["celltype"].isin(["CD4+ T"]),"celltype_num"] = "26"
COPD.obs.loc[COPD.obs["celltype"].isin(["CD8+ T"]),"celltype_num"] = "27"
COPD.obs.loc[COPD.obs["celltype"].isin(["Proliferating T"]),"celltype_num"] = "28"
COPD.obs.loc[COPD.obs["celltype"].isin(["NKT"]),"celltype_num"] = "29"
COPD.obs.loc[COPD.obs["celltype"].isin(["NK"]),"celltype_num"] = "30"
COPD.obs.loc[COPD.obs["celltype"].isin(["ILC"]),"celltype_num"] = "31"

COPD.obs.loc[COPD.obs["celltype"].isin(["cDC1"]),"celltype_num"] = "32"
COPD.obs.loc[COPD.obs["celltype"].isin(["cDC2"]),"celltype_num"] = "33"
COPD.obs.loc[COPD.obs["celltype"].isin(["Migratory DC"]),"celltype_num"] = "34"
COPD.obs.loc[COPD.obs["celltype"].isin(["pDC"]),"celltype_num"] = "35"
COPD.obs.loc[COPD.obs["celltype"].isin(["Classical monocyte"]),"celltype_num"] = "36"
COPD.obs.loc[COPD.obs["celltype"].isin(["Non-classical monocyte"]),"celltype_num"] = "37"
COPD.obs.loc[COPD.obs["celltype"].isin(["Alveolar macrophage"]),"celltype_num"] = "38"
COPD.obs.loc[COPD.obs["celltype"].isin(["Monocyte-derived macrophage"]),"celltype_num"] = "39"
COPD.obs.loc[COPD.obs["celltype"].isin(["Interstitial macrophage"]),"celltype_num"] = "40"
COPD.obs.loc[COPD.obs["celltype"].isin(["Mast"]),"celltype_num"] = "41"
COPD.obs.loc[COPD.obs["celltype"].isin(["Neutrophil"]),"celltype_num"] = "42"

In [ ]:
COPD.obs["celltype"] = COPD.obs["celltype"].astype(str)
COPD.obs["celltype_num"] = COPD.obs["celltype_num"].astype(str)

In [ ]:
COPD.obs["celltype_num_name"] = COPD.obs["celltype_num"] + "_" + COPD.obs["celltype"]

In [ ]:
sc.pl.umap(COPD, color=["celltype_num"], legend_loc="on data",legend_fontsize = 5, size = 1,
           palette=["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd", "#8c564b", "#e377c2", "#7f7f7f", "#bcbd22", "#17becf", 
                    "#6baed6", "#fd8d3c", "#74c476", "#9e9ac7", "#969696", "#5254a3", "#8ca252", "#bd9e39", "#ad494a", "#a55194", 
                    "#aec7e8", "#ffbb78", "#98df8a", "#ff9896", "#c5b0d5", "#c49c94", "#f786d2", "#c7c7c7", "#dbdb8d", "#9edae5", 
                     "#FF410D", "#6EE2FF", "#F7C530", "#95cc5e", "#d0dfe6", "#f79d1e", "#748aa6", "#cc0c00", "#5c88da", "#84bd00",
                     "#ffcd00", "#7c878e"],
          save = "COPD_newdata_celltype.pdf")

In [ ]:
sc.tl.rank_genes_groups(COPD, groupby="cellclass", method="wilcoxon")
marker = sc.get.rank_genes_groups_df(COPD,log2fc_min=0.5, group = ["Immature AT1","Mature AT1","AT1_AT2","AT2b","AT2s","RAS","Club","Basal","Goblet","Ciliated","PNEC",
"Adventitial fibroblast","Alveolar fibroblast","CTHRC1+ fibroblast","Myofibroblast","Fibromyocyte","Pericyte","ASM",
"VSM","Mesothelial","Lymphatic","Aerocyte","gCap","Arterial","Venous","Peribronchial","B","Plasma","Helper T","Treg",
"Cytotoxic T","NKT","NK","cDC1","cDC2","Migratory DC","pDC","Classical monocyte","Non-classical monocyte",
"Alveolar macrophage","Monocyte-derived macrophage","Interstitial macrophage","Mast","Neutrophil","ILC"])
marker = marker[marker.pvals_adj < 0.05]
marker.columns = ["group", "gene", "scores", "log2fc", "pvals", "pvals_adj"]
marker.to_csv("/datf/mazhuo/jupyter_notebook/COPD/tables/COPD_cellsubtype_marker.csv")

In [ ]:
COPD.obs.to_csv("/datf/mazhuo/jupyter_notebook/COPD/tables/COPD_meta_0808.csv")

In [ ]:
COPD.write("processed_data_py/COPD_newdata_0808_celltype.h5ad")

In [ ]:
COPD.obs['cellclass'] = pd.Categorical(COPD.obs['cellclass'], categories=["Epithelial","Mesenchymal","Mesothelial","Endothelial",
                                                                        "Myeloid","Lymphoid"], ordered=True)

In [ ]:
sc.pl.dotplot(COPD,{"Epithelial":["EPCAM","CDH1"],"Mesenchymal":["COL1A2","COL3A1","DCN"],
                    "Mesothelial":["UPK3B","WT1"],"Endothelial":["PECAM1","CDH5","CLDN5"],
                    "Immune":["PTPRC"],
                    "Myeloid":["ITGAM","FCGR3B"],
                    "Lymphoid":["CD3E","CD79B","NKG7"]}, 
                  groupby="cellclass",dendrogram=False,standard_scale="var", save = "COPD_cellclass_marker.pdf")

In [ ]:
COPD.obs['celltype'] = pd.Categorical(COPD.obs['celltype'], 
                                      categories=["Immature AT1","Mature AT1","AT2","TRB Secretory","PreTB Secretory","Basal","Goblet","Ciliated","PNEC",
"Adventitial fibroblast","Alveolar fibroblast","Myofibroblast","Fibromyocyte","Pericyte","ASM",
"VSM","Mesothelial","Lymphatic","Aerocyte","gCap","Arterial","Venous","B","Plasma","Treg","CD4+ T",
"CD8+ T","Proliferating T","NKT","NK","cDC1","cDC2","Migratory DC","pDC","Classical monocyte","Non-classical monocyte",
"Alveolar macrophage","Monocyte-derived macrophage","Interstitial macrophage","Mast","Neutrophil","Basophil"], ordered=True)

In [ ]:
sc.pl.dotplot(COPD,{"AT1":["AGER","HOPX","COL4A1","GPC5"], "AT2":["SFTPC","LAMP3","HHIP"], "Secretory":["SCGB3A2","SCGB1A1"], 
                   "Basal":["TP63","KRT5"], "Goblet":["MUC5B","BPIFB1"],"Ciliated":["FOXJ1","PIFO"], "PNEC":["CHGA","GRP"],
                   "Adventitial fibroblast": ["SERPINF1","SFRP2","PI16"], "Alveolar fibroblast" : ["GPC3","ITGA8","SPINT2"],
                   "Myofibroblast" : ["ASPN","ITGBL1","PDGFRA"], "Fibromyocyte" : ["ZNF385D","HPSE2","LGR6"],
                   "Pericyte": ["LAMC3","COX4I2"], "ASM" : ["TAGLN","ACTA2"], "VSM" : ["RGS5","SLIT3"], 
                   "Mesothelial":["UPK3B","WT1"], "Lymphatic": ["LYVE1","PROX1"], "gCap": ["FCN3","IL7R"],
                   "Aerocyte": ["EDNRB","HPGD"], "Arterial": ["IGFBP3","DKK2"],"Venous": ["CPE","HDAC9"],
                   "B": ["MS4A1","BCL11A","CD79A"], "Plasma": ["TNFRSF17","DERL3"],"T":["CD3E"],
                   "Treg": ["FOXP3","CTLA4"], "CD4+ T": ["CD4"], "CD8+ T": ["CD8A","CD8B"], 
                   "Proliferating": ["MKI67"],"NK":["NKG7","GNLY"],
                   "cDC1": ["CLEC9A","CLNK"], "cDC2": ["CLEC10A","CD1C","CD1E"],
                   "Migratory DC": ["LAD1","CCL19"], "pDC": ["SCT","SMPD3"],
                    "Monocyte":["FCN1","VCAN"],"Classical monocyte":["CD14"],"Non-classical monocyte":["FCGR3A"],
                   "Macrophage": ["CD68","MARCO"], "Alveolar macrophage": ["FABP4"], 
                   "Monocyte-derived macrophage":["TREM2"],"Interstitial macrophage": ["F13A1","FOLR2"],
                   "Mast": ["MS4A2"], "Neutrophil": ["FCGR3B"],
                   "Basophil": ["ENPP3","IL3RA"]}, swap_axes = True,
                  groupby="celltype",dendrogram=False,standard_scale="var", save = "COPD_celltype_marker.pdf")